In [1]:
from src.eval import main_mcnemar

ImportError: cannot import name 'main_mcnemar' from 'src.eval' (/home/semcovici/pesquisa/stance-prediction-UstanceBR/src/eval/__init__.py)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import classification_report
from transformers import set_seed
import os
import sys
from tqdm import tqdm 

tqdm.pandas()


sys.path.append('src/')
from data.lambdas import int_to_label, label_to_int
set_seed(42)

# Define paths (adaptado do seu código existente)
raw_data_path = 'data/raw/'
processed_data_path = 'data/processed/'
reports_path = 'reports/'
file_format_tmt = processed_data_path + "{split}_r3_{target}_top_mentioned_timelines_processed.csv"
file_format_users = processed_data_path + 'r3_{target}_{split}_users_processed.csv'
file_format_users_scored = processed_data_path + 'r3_{target}_{split}_users_scored_Timeline.csv'
file_format_tmt_scored = processed_data_path + '{split}_r3_{target}_top_mentioned_timelines_scored_Texts.csv'

# Target list
target_list = ['lu']

dict_exps = {
    "Timeline": {
        'path_dataset': file_format_users,
        "text_col": "Timeline",
        "batch_size": 64,
        "epochs": 1000
    },
    "Texts": {
        'path_dataset': file_format_tmt,
        "text_col": "Texts",
        "batch_size": 64,
        "epochs": 1000
    },
}

check_if_already_exists = False

for exp_name, config in dict_exps.items():
    
    print(f"""###########################################
# Running: {exp_name} 
###########################################""")
    
    text_col = config['text_col']
    path_dataset = config['path_dataset']
    
    # Processar cada target
    for target in target_list:
        
        print(f"""######## target: {target}""")
        estimator_name = 'LSTM'
        test_results_path = f"{reports_path}test_results/{estimator_name}_{target}_{exp_name}_test_results.csv"
        train_results_path = f"{reports_path}train_results/{estimator_name}_{target}_{exp_name}_train_results.csv"
        val_results_path = f"{reports_path}val_results/{estimator_name}_{target}_{exp_name}_val_results.csv"
        
        if os.path.isfile(test_results_path) and os.path.isfile(train_results_path) and check_if_already_exists:
            print('# experiment already done')
            continue
        
        # Ler e dividir os dados
        train_val = pd.read_csv(
            path_dataset.format(target=target, split="train"), 
            sep=';', 
            encoding='utf-8-sig'
        ).reset_index()[[text_col, 'Polarity']].rename(columns={text_col: 'text', 'Polarity': 'label'})
        
        train_val.label = train_val.label.apply(lambda x: label_to_int(x))
        
        # train_val.text = train_val.text.progress_apply(lambda x: x[:13000])

        # Check if label is binary
        if len(train_val.label.unique()) != 2:
            raise Exception("There is an error in train_val label transformation: expected to be binary")
        
        train, val = train_test_split(train_val, test_size=0.15, random_state=42)
        train.reset_index(drop=True, inplace=True)
        val.reset_index(drop=True, inplace=True)
        
        test = pd.read_csv(
            path_dataset.format(target=target, split="test"), 
            sep=';', 
            encoding='utf-8-sig'
        ).reset_index()[[text_col, 'Polarity']].rename(columns={text_col: 'text', 'Polarity': 'label'})
        
        test.label = test.label.apply(lambda x: label_to_int(x))
        input_tokens = 1000
        # Tokenize and pad texts
        tokenizer = Tokenizer(num_words=input_tokens)
        tokenizer.fit_on_texts(train['text'])
        
        X_train = tokenizer.texts_to_sequences(train['text'])
        X_val = tokenizer.texts_to_sequences(val['text'])
        X_test = tokenizer.texts_to_sequences(test['text'])
        
        max_length = max([len(x) for x in X_train])
        X_train = pad_sequences(X_train, maxlen=max_length)
        X_val = pad_sequences(X_val, maxlen=max_length)
        X_test = pad_sequences(X_test, maxlen=max_length)
        
        y_train = np.array(train['label'])
        y_val = np.array(val['label'])
        y_test = np.array(test['label'])
        
        # Build LSTM model
        model = Sequential([
            Embedding(input_dim=input_tokens, output_dim=128, input_length=max_length),
            LSTM(64, return_sequences=True),
            LSTM(32),
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Train the model
        model.fit(X_train, y_train, epochs=config['epochs'], batch_size=config['batch_size'], validation_data=(X_val, y_val))
        
        # Evaluate the model
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)
        test_pred = model.predict(X_test)
        
        # Convert probabilities to binary predictions
        train_pred = (train_pred > 0.5).astype(int)
        val_pred = (val_pred > 0.5).astype(int)
        test_pred = (test_pred > 0.5).astype(int)